In [7]:
# Imports, DB path, & Helper to convert rows → dicts

import sqlite3
import uuid
from backend import (
    create_incident,
    read_incidents,
    search_incident,
    update_incident,
    delete_incident
)

DB_PATH = "database/911_Call_Data.db"
print(f"Using database at: {DB_PATH}")

# Fetch column names from the Calls table
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
cursor.execute("PRAGMA table_info(Calls)")
cols = [colinfo[1] for colinfo in cursor.fetchall()]  # e.g. ["call_key","record_id",...]
conn.close()

def row_to_dict(row):
    """
    Given either a sqlite3.Row or a tuple, return a dict
    mapping column names → values.
    """
    return dict(zip(cols, row))

print("Columns:", cols)
print("row_to_dict([1,2,3]) =", row_to_dict([1, 2, 3]))

Using database at: database/911_Call_Data.db
Columns: ['call_key', 'record_id', 'call_date_time', 'priority', 'description', 'call_number', 'incident_location_id', 'reporter_location_id', 'jurisdiction_id']
row_to_dict([1,2,3]) = {'call_key': 1, 'record_id': 2, 'call_date_time': 3}


In [8]:
# Test create_incident

test_record_id    = str(uuid.uuid4())
test_time         = "2025/05/03 00:00:00+00"  # UTC format
test_priority     = "High"
test_description  = f"TEST INCIDENT at {uuid.uuid4()}"
test_call_number  = "TEST123"
incident_loc_id   = 1
reporter_loc_id   = 1
jurisdiction_id   = 1

new_call_key = create_incident(
    record_id=test_record_id,
    call_date_time=test_time,
    priority=test_priority,
    description=test_description,
    call_number=test_call_number,
    incident_location_id=incident_loc_id,
    reporter_location_id=reporter_loc_id,
    jurisdiction_id=jurisdiction_id
)

assert isinstance(new_call_key, str) and len(new_call_key) > 0
print("create_incident returned call_key:", new_call_key)


create_incident returned call_key: d56624a4-4604-4fa9-bf9e-fe8ee9b95f90


In [9]:
# Test read_incidents

latest_rows = read_incidents()      # returns up to 10 rows
latest = [row_to_dict(r) for r in latest_rows]

assert any(d["call_key"] == new_call_key for d in latest), (
    "❌ Our test incident was not found in read_incidents()"
)
print("read_incidents found our new incident.")


read_incidents found our new incident.


In [10]:
# Test search_incident

search_rows = search_incident(new_call_key)
assert isinstance(search_rows, list), "search_incident must return a list"
assert len(search_rows) == 1, f"Expected 1 result, got {len(search_rows)}"

record = row_to_dict(search_rows[0])
assert record["description"] == test_description, (
    f"Expected description {test_description}, got {record['description']}"
)
assert record["priority"] == test_priority, (
    f"Expected priority {test_priority}, got {record['priority']}"
)
assert record["call_number"] == test_call_number, (
    f"Expected call_number {test_call_number}, got {record['call_number']}"
)

print("search_incident returned the correct record.")


search_incident returned the correct record.


In [11]:
# Test update_incident

new_priority    = "Medium"
new_description = test_description + " [UPDATED]"

# update_incident only needs call_key + any fields to change
update_incident(
    call_key=new_call_key,
    priority=new_priority,
    description=new_description
)

# fetch again
updated_rows = search_incident(new_call_key)
assert len(updated_rows) == 1, "After update, expected exactly 1 result"
updated = row_to_dict(updated_rows[0])

assert updated["priority"] == new_priority, (
    f"Priority did not update: got {updated['priority']}"
)
assert updated["description"] == new_description, (
    f"Description did not update: got {updated['description']}"
)

print("update_incident successfully modified the record.")


update_incident successfully modified the record.


In [12]:
# Test delete_incident

delete_incident(new_call_key)

post_delete = search_incident(new_call_key)
assert isinstance(post_delete, list), "search_incident should still return a list"
assert len(post_delete) == 0, "Record was not deleted"

print("delete_incident removed our test record.")


delete_incident removed our test record.


In [13]:
# Final read sanity

_ = read_incidents()
print("Final read_incidents sanity check passed.")


Final read_incidents sanity check passed.
